## Single Allocation p-Hub Location Problem (SApHLP)
Uncapacitated single allocation p-hub location problem on CAB,TR and randomly generated datasets to minimize the total transportation cost and allocated demand to these hubs.

Assumptions: 
• Every origin-destination path visits at least 1 hub.
• Inter-hub cost per unit flow is discounted using α, discount factor.
• Facilities do not fail.

In [ ]:
# Importing packages
import pandas as pd
import numpy as np
import random as rd
from itertools import combinations
import copy
import math
import time

In [ ]:
# Random initial solution generation
def initial_solution(nodes):
    hubs = rd.sample(nodes,p)
    init_sol = [hubs[rd.randint(0,p-1)] for i in range(len(nodes))]
    for hub in hubs:
        init_sol[hub] = hub #to allocate the hubs to themselves for feasibility
    return init_sol

In [ ]:
init_sol = initial_solution(nodes)

##### Objective Function

In [ ]:
def total_cost(solution):
    total_cost = 0
    for i in range(0,len(nodes)):
        for j in range(0, len(nodes)): 
            total_cost += flow[i][j]*(cost[i][solution[i]]+discount_factor*cost[solution[i]][solution[j]]+cost[solution[j]][j])
    return total_cost/total_flow

In [ ]:
total_cost(init_sol)

##### Neighborhood Structures

In [ ]:
# A function for when a hub changes, all nodes allocated to it must be updated with the new hub - using nodes' indices
# Feasibility by avoiding more than 3 hubs and keping hubs allocated to themselves is provided within neighborhood st.s
def change_hub(solution, indices, new_hub):
    new_solution = solution.copy()
    for index in indices:
        new_solution[index] = new_hub
    return new_solution

In [ ]:
def neigh_1(solution): # change hub with a node allocated to it
    neighborhood_st = []
    for hub in set(solution): # getting list of indices of nodes allocated to that hub
        alloc_ind = [ind for ind in range(len(solution)) if solution[ind] == hub] 
        for ind in alloc_ind:
            if ind == hub: # keeping hubs allocated to themselves
                continue
            new_solution = change_hub(solution, alloc_ind, ind)
            neighborhood_st.append(new_solution)
    return neighborhood_st

In [ ]:
def neigh_2(solution): # change two nodes' hubs with each other keeping the hubs same
    neighborhood_st = []
    for i in range(len(solution)):
        if i == solution[i]:
            continue
        for j in range(0, len(solution)):
            if j == solution[j]:
                continue
            if solution[i] == solution[j]:
                continue
            new_solution = solution.copy()
            new_solution[i], new_solution[j] = new_solution[j], new_solution[i]
            neighborhood_st.append(new_solution)
    return neighborhood_st

In [ ]:
def neigh_3(solution): # changing nodes' hub to another keeping the hubs same
    neighborhood_st = []
    for i in range(len(solution)):
        if i == solution[i]:
            continue
        hubs = list(set(solution))
        hubs.remove(solution[i])
        new_solution = solution.copy()
        for h in hubs:
            indices = [i]
            new_solution = change_hub(solution, indices, h)
            neighborhood_st.append(new_solution)
    return neighborhood_st

In [ ]:
def neigh_4(solution): # changing allocated node groups between hubs keeping the hubs same
    neighborhood_st = []
    hubs = list(set(solution))
    allocations = []
    for hub in set(solution):
        alloc_ind = [ind for ind in range(len(solution)) if solution[ind] == hub and solution[ind] != ind] # getting list of indices of nodes allocated to that hub
        allocations.append(alloc_ind)
    new_solution = solution.copy()
    for i in allocations[0]:
        new_solution[i] = hubs[1] # iterate these indices too?
        for j in allocations[1]:
            if solution[j] == hubs[1]:
                new_solution[j] = hubs[0]
    neighborhood_st.append(new_solution)
    return neighborhood_st

##### Simulated Annealing

In [ ]:
from math import exp

In [ ]:
def simulated_annealing(objective, n_iterations, temp):
    init_sol = initial_solution(nodes)
    best = init_sol
    best_eval = total_cost(init_sol)
    # current candidate solution
    current, current_eval = best, best_eval
    # starting iterations by passing to a new candidate from current sol at each step
    for i in range(n_iterations):
        candidate = mutation(current, 1) # using GA's mutation function below to include all neighborhood structures built
        candidate_eval = total_cost(candidate)
        if candidate_eval < best_eval:
            best, best_eval = candidate, candidate_eval
            print(f"New best! Total cost = {best_eval} Allocation: {best} Iteration: {i}")
        diff = candidate_eval - current_eval
        # cooling temperature
        t = temp / float(i+1) # i+1 to avoid 0 just in case
        # transition (acceptance) probability (from current to candidate)
        prob = exp(-diff / t)
        # checking to have transition or not
        if diff < 0 or rd.random() < prob:
            current, current_eval = candidate, candidate_eval
    return [best, best_eval]

##### Genetic Algorithm

In [ ]:
# Initial population and Hyperparameters
# size of population
#n_pop = 100 
# initial population
#pop = [list(initial_solution(nodes)) for _ in range(n_pop)]
# number of generations = iterations
#n_gen = 30
# crossover rate
r_cross = 0.9
# mutation rate
r_mut = 0.1

In [ ]:
# Tournament selection 
def tournament_selection(pop, k):
    ind_group1 = rd.sample(pop, k)
    scores = [total_cost(ind) for ind in ind_group1]
    selection = rd.choice(ind_group1)
    for ind in ind_group1:
        if total_cost(ind) < total_cost(selection):
            selection = ind
    p1 = selection
    
    ind_group2 = rd.sample(pop, k)
    scores2 = [total_cost(ind) for ind in ind_group2]
    selection2 = rd.choice(ind_group2)
    for ind in ind_group2:
        if total_cost(ind) < total_cost(selection2):
            selection2 = ind
    p2 = selection2
    
    parents = [p1, p2]
    
    return parents

In [ ]:
# Crossover (two parents for two children)
def crossover(p1, p2, r_cross):
    c1, c2 = p1.copy(), p2.copy()
    
    if rd.random() < r_cross: # randomly deciding whether crossover takes place, r_cross is high to make it happen often
        point = rd.randint(1, len(p1)-2) # selecting crossover point in p1, 1 to len-2 to avoid beginning or end of sol.
        c1 = p1[:point] + p2[point:]
        c2 = p2[:point] + p1[point:]
     
    # continue with mutant parent if a child is infeasible
    if len(set(c1)) > p:
        c1 = mutation(p1, 1)
    if len(set(c2)) > p:
        c2 = mutation(p2, 1)
    
    for c in c1:
        for i in list(set(c1)):
            if c1[i] != i:
                c1 = mutation(p1, 1)
    for c in c2:
        for i in list(set(c2)):
            if c2[i] != i:
                c2 = mutation(p2, 1)          
    
    return [c1, c2]

In [ ]:
# Mutation (for a child)
def mutation(child, r_mut):  
    mutant = child.copy()
    if rd.random() < r_mut: # randomly deciding for a mutation or not
        mutations = ["NS1", "NS2", "NS3", "NS4"] # neighborhood functions also provide feasibility :)
        chosen_mutation = rd.choice(mutations)
        #print(chosen_mutation)
        if chosen_mutation == "NS1":
            mutant = rd.choice(neigh_1(child)) 
        elif chosen_mutation == "NS2":
            mutant = rd.choice(neigh_2(child))
        elif chosen_mutation == "NS3":
            mutant = rd.choice(neigh_3(child))
        else:
            mutant = rd.choice(neigh_4(child))
    return mutant 

In [ ]:
# Creating Generations
def genetic_algorithm(objective, n_gen, n_pop, r_cross, r_mut):
    # initial population - have different initial pop each time GA runs
    pop = [list(initial_solution(nodes)) for _ in range(n_pop)]
    # tracking best solution
    best, best_score = 0, total_cost(pop[0])
    # enumarating generations
    for gen in range(n_gen):
        scores = [total_cost(c) for c in pop] # evaluating all candidates in pop
        # checking for best solution
        for i in range(len(pop)):
            #print(f"Generation {gen} Total cost = {scores[i]} Allocation: {pop[i]}")
            if scores[i] < best_score:
                best, best_score = pop[i], scores[i]
                print(f"Generation {gen} New best! : Total cost = {best_score} Allocation: {best}")
        # tournament selection to get parents list
        parents = tournament_selection(pop, 4)
        # creating next generation
        children = []
        for i in range(n_pop):
            p1, p2 = parents[0], parents[1] 
            for c in crossover(p1, p2, r_cross): # crossover & mutation
                mutation(c, r_mut)
                children.append(c)
        # making new generation the next population
        pop = children
    return [best, best_score]

##### Implementations

In [ ]:
# CAB25
print("CAB 25")
flow = pd.read_excel('CAB_TR_RGP_Datasets 2022.xlsx', sheet_name = 'CAB25Flow', header = None)
total_flow = flow.sum().sum()
cost = pd.read_excel('CAB_TR_RGP_Datasets 2022.xlsx', sheet_name = 'CAB25Cost', header = None)
nodes = list(flow.columns)

In [ ]:
p = 3
discount_factor = 0.2
temp = 50
n_iterations = 300
n_pop = 100
n_gen = 20

In [ ]:
cab25_SA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = simulated_annealing(total_cost, n_iterations, temp)
    print(f"Done! SA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    cab25_SA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
cab25_SA

In [ ]:
sum(cab25_SA)/len(cab25_SA)

In [ ]:
# CAB25 GA
cab25_GA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = genetic_algorithm(total_cost, n_pop, n_gen, 0.9, 0.1)
    print(f"Done! GA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    cab25_GA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
cab25_GA

In [ ]:
sum(cab25_GA)/len(cab25_GA)

In [ ]:
p = 3
discount_factor = 0.8
temp = 50
n_iterations = 300
n_pop = 100
n_gen = 20

In [ ]:
cab25_SA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = simulated_annealing(total_cost, n_iterations, temp)
    print(f"Done! SA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    cab25_SA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
cab25_SA

In [ ]:
sum(cab25_SA)/len(cab25_SA)

In [ ]:
# CAB25 GA
cab25_GA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = genetic_algorithm(total_cost, n_pop, n_gen, 0.9, 0.1)
    print(f"Done! GA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    cab25_GA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
cab25_GA

In [ ]:
sum(cab25_GA)/len(cab25_GA)

In [ ]:
p = 5
discount_factor = 0.2
temp = 60
n_iterations = 500
n_pop = 100
n_gen = 30

In [ ]:
cab25_SA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = simulated_annealing(total_cost, n_iterations, temp)
    print(f"Done! SA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    cab25_SA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
cab25_SA

In [ ]:
sum(cab25_SA)/len(cab25_SA)

In [ ]:
# CAB25 GA
cab25_GA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = genetic_algorithm(total_cost, n_pop, n_gen, 0.9, 0.1)
    print(f"Done! GA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    cab25_GA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
cab25_GA

In [ ]:
sum(cab25_GA)/len(cab25_GA)

In [ ]:
p = 5
discount_factor = 0.8
temp = 50
n_iterations = 500
n_pop = 100
n_gen = 30

In [ ]:
cab25_SA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = simulated_annealing(total_cost, n_iterations, temp)
    print(f"Done! SA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    cab25_SA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
cab25_SA

In [ ]:
sum(cab25_SA)/len(cab25_SA)

In [ ]:
# CAB25 GA
cab25_GA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = genetic_algorithm(total_cost, 50, 20, 0.9, 0.1)
    print(f"Done! GA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    cab25_GA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
cab25_GA

In [ ]:
sum(cab25_GA)/len(cab25_GA)

In [ ]:
# TR55
print("TR 55")
flow = pd.read_excel('CAB_TR_RGP_Datasets 2022.xlsx', sheet_name = 'TR55Flow', header = None)
total_flow = flow.sum().sum()
cost = pd.read_excel('CAB_TR_RGP_Datasets 2022.xlsx', sheet_name = 'TR55Cost', header = None)
nodes = list(flow.columns)

In [ ]:
p = 3
discount_factor = 0.2
temp = 50
n_iterations = 600
n_pop = 100
n_gen = 30

In [ ]:
# TR55 SA
tr55_SA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = simulated_annealing(total_cost, n_iterations, temp)
    print(f"Done! SA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    tr55_SA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
tr55_SA

In [ ]:
sum(tr55_SA)/len(tr55_SA)

In [ ]:
tr55_GA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = genetic_algorithm(total_cost, n_pop, n_gen, 0.9, 0.1)
    print(f"Done! GA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    cab25_GA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
tr55_GA

In [ ]:
sum(tr55_GA)/len(tr55_GA)

In [ ]:
p = 3
discount_factor = 0.8
temp = 50
n_iterations = 600
n_pop = 100
n_gen = 30

In [ ]:
tr55_SA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = simulated_annealing(total_cost, n_iterations, temp)
    print(f"Done! SA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    tr55_SA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
tr55_SA

In [ ]:
sum(tr55_SA)/len(tr55_SA)

In [ ]:
tr55_GA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = genetic_algorithm(total_cost, n_pop, n_gen, 0.9, 0.1)
    print(f"Done! GA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    cab25_GA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
tr55_GA

In [ ]:
sum(tr55_GA)/len(tr55_GA)

In [ ]:
p = 5
discount_factor = 0.2
temp = 50
n_iterations = 1000
n_pop = 100
n_gen = 40

In [ ]:
tr55_SA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = simulated_annealing(total_cost, n_iterations, temp)
    print(f"Done! SA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    tr55_SA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
tr55_SA

In [ ]:
sum(tr55_SA)/len(tr55_SA)

In [ ]:
tr55_GA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = genetic_algorithm(total_cost, n_pop, n_gen, 0.9, 0.1)
    print(f"Done! GA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    cab25_GA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
tr55_GA

In [ ]:
sum(tr55_GA)/len(tr55_GA)

In [ ]:
p = 5
discount_factor = 0.8
temp = 50
n_iterations = 1000
n_pop = 100
n_gen = 40

In [ ]:
tr55_SA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = simulated_annealing(total_cost, n_iterations, temp)
    print(f"Done! SA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    tr55_SA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
tr55_SA

In [ ]:
sum(tr55_SA)/len(tr55_SA)

In [ ]:
tr55_GA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = genetic_algorithm(total_cost, n_pop, n_gen, 0.9, 0.1)
    print(f"Done! GA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    cab25_GA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
tr55_GA

In [ ]:
sum(tr55_GA)/len(tr55_GA)

In [ ]:
# TR81
print("TR 81")
flow = pd.read_excel('CAB_TR_RGP_Datasets 2022.xlsx', sheet_name = 'TR81Flow', header = None)
total_flow = flow.sum().sum()
cost = pd.read_excel('CAB_TR_RGP_Datasets 2022.xlsx', sheet_name = 'TR81Cost', header = None)
nodes = list(flow.columns)

In [ ]:
p = 5
discount_factor = 0.2
temp = 50
n_iterations = 800
n_pop = 100
n_gen = 40

In [ ]:
# TR81 SA
tr81_SA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = simulated_annealing(total_cost, n_iterations, temp)
    print(f"Done! SA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    tr81_SA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
tr81_SA

In [ ]:
sum(tr81_SA)/len(tr81_SA)

In [ ]:
tr81_GA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = genetic_algorithm(total_cost, n_pop, n_gen, 0.9, 0.1)
    print(f"Done! GA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    cab25_GA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
tr81_GA

In [ ]:
sum(tr81_GA)/len(tr81_GA)

In [ ]:
p = 5
discount_factor = 0.8
temp = 50
n_iterations = 800
n_pop = 100
n_gen = 40

In [ ]:
tr81_SA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = simulated_annealing(total_cost, n_iterations, temp)
    print(f"Done! SA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    tr81_SA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
tr81_SA

In [ ]:
sum(tr81_SA)/len(tr81_SA)

In [ ]:
tr81_GA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = genetic_algorithm(total_cost, n_pop, n_gen, 0.9, 0.1)
    print(f"Done! GA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    cab25_GA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
tr81_GA

In [ ]:
sum(tr81_GA)/len(tr81_GA)

In [ ]:
p = 7
discount_factor = 0.2
temp = 50
n_iterations = 1000
n_pop = 100
n_gen = 50

In [ ]:
tr81_SA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = simulated_annealing(total_cost, n_iterations, temp)
    print(f"Done! SA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    tr81_SA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
tr81_SA

In [ ]:
sum(tr81_SA)/len(tr81_SA)

In [ ]:
tr81_GA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = genetic_algorithm(total_cost, n_pop, n_gen, 0.9, 0.1)
    print(f"Done! GA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    cab25_GA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
tr81_GA

In [ ]:
sum(tr81_GA)/len(tr81_GA)

In [ ]:
p = 7
discount_factor = 0.8
temp = 50
n_iterations = 1000
n_pop = 100
n_gen = 50

In [ ]:
tr81_SA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = simulated_annealing(total_cost, n_iterations, temp)
    print(f"Done! SA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    tr81_SA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
tr81_SA

In [ ]:
sum(tr81_SA)/len(tr81_SA)

In [ ]:
tr81_GA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = genetic_algorithm(total_cost, n_pop, n_gen, 0.9, 0.1)
    print(f"Done! GA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    cab25_GA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
tr81_GA

In [ ]:
sum(tr81_GA)/len(tr81_GA)

In [ ]:
# RGP100
print("RGP 100")
flow = pd.read_excel('CAB_TR_RGP_Datasets 2022.xlsx', sheet_name = 'RGP100Flow', header = None)
total_flow = flow.sum().sum()
cost = pd.read_excel('CAB_TR_RGP_Datasets 2022.xlsx', sheet_name = 'RGP100Cost', header = None)
nodes = list(flow.columns)

In [ ]:
p = 7
discount_factor = 0.2
temp = 50
n_iterations = 1200
n_pop = 100
n_gen = 50

In [ ]:
# RGP100 SA
rgp100_SA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = simulated_annealing(total_cost, n_iterations, temp)
    print(f"Done! SA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    rgp100_SA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
rgp100_SA

In [ ]:
sum(rgp100_SA)/len(rgp100_SA)

In [ ]:
rgp100_GA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = genetic_algorithm(total_cost, n_pop, n_gen, 0.9, 0.1)
    print(f"Done! GA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    cab25_GA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
rgp100_GA

In [ ]:
sum(rgp100_GA)/len(rgp100_GA)

In [ ]:
p = 7
discount_factor = 0.8
temp = 50
n_iterations = 1200
n_pop = 100
n_gen = 50

In [ ]:
rgp100_SA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = simulated_annealing(total_cost, n_iterations, temp)
    print(f"Done! SA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    rgp100_SA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
rgp100_SA

In [ ]:
sum(rgp100_SA)/len(rgp100_SA)

In [ ]:
rgp100_GA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = genetic_algorithm(total_cost, n_pop, n_gen, 0.9, 0.1)
    print(f"Done! GA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    cab25_GA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
rgp100_GA

In [ ]:
sum(rgp100_GA)/len(rgp100_GA)

In [ ]:
p = 10
discount_factor = 0.2
temp = 50
n_iterations = 1500
n_pop = 100
n_gen = 60

In [ ]:
rgp100_SA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = simulated_annealing(total_cost, n_iterations, temp)
    print(f"Done! SA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    rgp100_SA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
rgp100_SA

In [ ]:
sum(rgp100_SA)/len(rgp100_SA)

In [ ]:
rgp100_GA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = genetic_algorithm(total_cost, n_pop, n_gen, 0.9, 0.1)
    print(f"Done! GA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    cab25_GA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
rgp100_GA

In [ ]:
sum(rgp100_GA)/len(rgp100_GA)

In [ ]:
p = 10
discount_factor = 0.8
temp = 50
n_iterations = 1500
n_pop = 100
n_gen = 60

In [ ]:
rgp100_SA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = simulated_annealing(total_cost, n_iterations, temp)
    print(f"Done! SA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    rgp100_SA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
rgp100_SA

In [ ]:
sum(rgp100_SA)/len(rgp100_SA)

In [ ]:
rgp100_GA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = genetic_algorithm(total_cost, n_pop, n_gen, 0.9, 0.1)
    print(f"Done! GA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    cab25_GA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
rgp100_GA

In [ ]:
sum(rgp100_GA)/len(rgp100_GA)

In [ ]:
# RGP130
print("RGP 130")
flow = pd.read_excel('CAB_TR_RGP_Datasets 2022.xlsx', sheet_name = 'RGP130Flow', header = None)
total_flow = flow.sum().sum()
cost = pd.read_excel('CAB_TR_RGP_Datasets 2022.xlsx', sheet_name = 'RGP130Cost', header = None)
nodes = list(flow.columns)

In [ ]:
p = 7
discount_factor = 0.2
temp = 50
n_iterations = 1500
n_pop = 100
n_gen = 60

In [ ]:
# RGP130 SA
rgp130_SA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = simulated_annealing(total_cost, n_iterations, temp)
    print(f"Done! SA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    rgp130_SA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
rgp130_SA

In [ ]:
sum(rgp130_SA)/len(rgp130_SA)

In [ ]:
rgp130_GA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = genetic_algorithm(total_cost, n_pop, n_gen, 0.9, 0.1)
    print(f"Done! GA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    cab25_GA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
rgp130_GA

In [ ]:
sum(rgp130_GA)/len(rgp130_GA)

In [ ]:
p = 7
discount_factor = 0.8
temp = 50
n_iterations = 1500
n_pop = 100
n_gen = 60

In [ ]:
rgp130_SA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = simulated_annealing(total_cost, n_iterations, temp)
    print(f"Done! SA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    rgp130_SA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
rgp130_SA

In [ ]:
sum(rgp130_SA)/len(rgp130_SA)

In [ ]:
rgp130_GA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = genetic_algorithm(total_cost, n_pop, n_gen, 0.9, 0.1)
    print(f"Done! GA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    cab25_GA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
rgp130_GA

In [ ]:
sum(rgp130_GA)/len(rgp130_GA)

In [ ]:
p = 10
discount_factor = 0.2
temp = 50
n_iterations = 1800
n_pop = 100
n_gen = 70

In [ ]:
rgp130_SA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = simulated_annealing(total_cost, n_iterations, temp)
    print(f"Done! SA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    rgp130_SA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
rgp130_SA

In [ ]:
sum(rgp130_SA)/len(rgp130_SA)

In [ ]:
rgp130_GA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = genetic_algorithm(total_cost, n_pop, n_gen, 0.9, 0.1)
    print(f"Done! GA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    cab25_GA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
rgp130_GA

In [ ]:
sum(rgp130_GA)/len(rgp130_GA)

In [ ]:
p = 10
discount_factor = 0.8
temp = 50
n_iterations = 1800
n_pop = 100
n_gen = 70

In [ ]:
rgp130_SA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = simulated_annealing(total_cost, n_iterations, temp)
    print(f"Done! SA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    rgp130_SA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
rgp130_SA

In [ ]:
sum(rgp130_SA)/len(rgp130_SA)

In [ ]:
rgp130_GA = []
for i in range(10):
    start_time = time.time()
    best, best_eval = genetic_algorithm(total_cost, n_pop, n_gen, 0.9, 0.1)
    print(f"Done! GA-run[{i}] Total cost = {best_eval} Allocation: {best}")
    cab25_GA.append(best_eval)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

In [ ]:
rgp130_GA

In [ ]:
sum(rgp130_GA)/len(rgp130_GA)